In [2]:
import numpy as np
from tensorflow import keras
from keras.datasets import mnist
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report 


In [3]:
# Tải dữ liệu huấn luyện và dữ liệu đánh giá cho mô hình
(x_train, y_train),(x_test, y_test)= mnist.load_data()

In [21]:
# xây dựng cấu trúc mô hình
from tensorflow import keras
def DCNN1_model():
    #CNN layer
    model = keras.Sequential()
    model.add(keras.layers.Conv2D(64,(5,5), input_shape=(28,28,1), activation = 'relu'))
    model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))
    model.add(keras.layers.Dropout(0,25))
    model.add(keras.layers.Conv2D(32, (3,3),activation='relu'))
    model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))
    model.add(keras.layers.Dropout(0,25))

    model.add(keras.layers.Flatten())

    # MLP with 3 hidden layer
    model.add(keras.layers.Dense(375, activation='relu'))
    model.add(keras.layers.Dropout(0.25))
    model.add(keras.layers.Dense(225, activation='relu'))
    model.add(keras.layers.Dropout(0.25))
    model.add(keras.layers.Dense(135, activation='relu'))
    model.add(keras.layers.Dropout(0.25))
    model.add(keras.layers.Dense(10, activation='softmax'))
    model.compile(loss = 'sparse_categorical_crossentropy',optimizer = keras.optimizers.Adam(learning_rate=0.01), metrics=['accuracy'])
    return model

In [22]:
# huấn luyện mô hình
model = DCNN1_model()
model.fit(x_train, y_train, validation_data = (x_test, y_test), epochs=10, batch_size=256, verbose=2)


Epoch 1/10
235/235 - 40s - loss: 3.0346 - accuracy: 0.6142 - val_loss: 0.3185 - val_accuracy: 0.9070 - 40s/epoch - 171ms/step
Epoch 2/10
235/235 - 37s - loss: 0.3639 - accuracy: 0.9031 - val_loss: 0.2004 - val_accuracy: 0.9426 - 37s/epoch - 156ms/step
Epoch 3/10
235/235 - 35s - loss: 0.2885 - accuracy: 0.9239 - val_loss: 0.1747 - val_accuracy: 0.9521 - 35s/epoch - 150ms/step
Epoch 4/10
235/235 - 36s - loss: 0.2572 - accuracy: 0.9328 - val_loss: 0.1561 - val_accuracy: 0.9569 - 36s/epoch - 154ms/step
Epoch 5/10
235/235 - 35s - loss: 0.2690 - accuracy: 0.9317 - val_loss: 0.1741 - val_accuracy: 0.9490 - 35s/epoch - 150ms/step
Epoch 6/10
235/235 - 36s - loss: 0.2491 - accuracy: 0.9366 - val_loss: 0.1672 - val_accuracy: 0.9563 - 36s/epoch - 152ms/step
Epoch 7/10
235/235 - 36s - loss: 0.2505 - accuracy: 0.9373 - val_loss: 0.1578 - val_accuracy: 0.9576 - 36s/epoch - 152ms/step
Epoch 8/10
235/235 - 35s - loss: 0.2460 - accuracy: 0.9381 - val_loss: 0.1701 - val_accuracy: 0.9535 - 35s/epoch - 149

In [29]:
#lưu lại mô hình
model.save('hwr.h5')

d:\handwritten\env\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [28]:
#tải mô hình sẵn có
model2 = keras.models.load_model("hwr.h5")

In [25]:
# đánh giá mô hình 
pred = np.argmax(model2.predict(x_test),axis=1)
print(classification_report(pred,y_test))

313/313 [==============================] - 3s 10ms/step
              precision    recall  f1-score   support

           0       0.97      0.97      0.97       983
           1       0.98      0.99      0.98      1123
           2       0.95      0.97      0.96      1019
           3       0.98      0.94      0.96      1051
           4       0.93      0.97      0.95       947
           5       0.97      0.96      0.96       903
           6       0.97      0.98      0.97       953
           7       0.94      0.94      0.94      1019
           8       0.89      0.96      0.93       907
           9       0.97      0.90      0.93      1095

    accuracy                           0.96     10000
   macro avg       0.96      0.96      0.96     10000
weighted avg       0.96      0.96      0.96     10000



In [30]:
import tkinter as tk
from PIL import ImageGrab
import numpy as np
import tensorflow as tf
from tensorflow import keras

#tải mô hình sẵn có
model.save('hwr.h5')
# Hàm dự đoán chữ số
def predict_digit(image):
    image = image.resize((28, 28))
    image = image.convert('L')  # Đổi sang ảnh grayscale
    image = np.array(image)
    image = image.reshape(1, 28, 28, 1)
    image = image / 255.0
    prediction = model.predict(image)
    return np.argmax(prediction)

# Tạo giao diện
class App(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Handwritten Digit Recognition")
        self.canvas = tk.Canvas(self, width=200, height=200, bg='white')
        self.canvas.pack()
        
        self.button_predict = tk.Button(self, text="Predict", command=self.predict)
        self.button_predict.pack()
        
        self.button_save = tk.Button(self, text="Save Image", command=self.save_image)
        self.button_save.pack()

        self.button_clear = tk.Button(self, text="Clear", command=self.clear_canvas)
        self.button_clear.pack()
        
        self.label_result = tk.Label(self, text="Prediction: None", font=("Helvetica", 24))
        self.label_result.pack()

        self.canvas.bind("<B1-Motion>", self.draw)

    def draw(self, event):
        x, y = event.x, event.y
        self.canvas.create_oval(x - 5, y - 5, x + 5, y + 5, fill='black')

    def predict(self):
        x = self.winfo_rootx() + self.canvas.winfo_x()
        y = self.winfo_rooty() + self.canvas.winfo_y()
        x1 = x + self.canvas.winfo_width()
        y1 = y + self.canvas.winfo_height()
        image = ImageGrab.grab().crop((x, y, x1, y1))
        
        # Lưu ảnh để kiểm tra
        image.save("canvas_image.png")

        # Nhận diện chữ số
        digit = predict_digit(image)
        self.label_result.config(text=f"Prediction: {digit}")

    def save_image(self):
        x = self.winfo_rootx() + self.canvas.winfo_x()
        y = self.winfo_rooty() + self.canvas.winfo_y()
        x1 = x + self.canvas.winfo_width()
        y1 = y + self.canvas.winfo_height()
        image = ImageGrab.grab().crop((x, y, x1, y1))
        image = image.resize((28, 28)).convert("L")  # Đảm bảo ảnh giống với tập huấn luyện
        image.save("saved_canvas_image.png")
        print("Image saved as 'saved_canvas_image.png'")

    def clear_canvas(self):
        self.canvas.delete("all")
# Chạy ứng dụng
app = App()
app.mainloop()

1/1 [==============================] - 0s 18ms/step
